# Tutorial 09: Error Handling & Recovery

##  Learning Objectives
By the end of this tutorial, you will:
- Implement **retry logic** for transient failures
- Create **fallback strategies** when operations fail
- Build **circuit breakers** to prevent cascading failures
- Use **error boundaries** to contain failures
- Implement **graceful degradation** for partial failures
- Log and monitor errors for observability
- Understand **when to retry vs fail fast**

##  Prerequisites

Before starting this tutorial, you should have completed:
- **Tutorial 06: Multi-Agent Workflows** (Sequential, Concurrent patterns)
- **Tutorial 07: Advanced Workflows** (WorkflowBuilder, Custom Executors)
- **Tutorial 08: Human-in-the-Loop** (Approval gates, Interactive workflows)

##  Why Error Handling Matters

In production AI systems, things WILL go wrong:
- 🔴 **API Rate Limits** - Azure OpenAI throttles requests
- 🔴 **Network Timeouts** - Temporary connectivity issues
- 🔴 **Model Failures** - Invalid responses or context limits
- 🔴 **Resource Exhaustion** - Out of memory, quota exceeded
- 🔴 **External Dependencies** - Third-party APIs down

### Error Handling Strategies

| Strategy | When to Use | Example |
|----------|-------------|---------|
| **Retry** | Transient failures (network, rate limits) | Exponential backoff |
| **Fallback** | Service degradation | Use simpler model, cached data |
| **Circuit Breaker** | Cascading failures | Stop calling failing service |
| **Timeout** | Hanging operations | Set max execution time |
| **Graceful Degradation** | Partial failures | Return partial results |
| **Fail Fast** | Invalid input, logic errors | Immediate exception |

---

## Step 1: Setup and Imports

In [1]:
import asyncio
from typing import Optional, Any
from datetime import datetime, timedelta
import random
from enum import Enum

from agent_framework import (
    # Workflow builders
    WorkflowBuilder,
    SequentialBuilder,
    # Core components
    Executor,
    AgentExecutor,
    handler,
    WorkflowContext,
    # Events
    WorkflowOutputEvent,
    # Message types
    ChatMessage,
    Role,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

load_dotenv()
print(" Imports successful!")
print("🛡 Ready to build resilient AI systems!")

 Imports successful!
🛡 Ready to build resilient AI systems!


## Step 2: Pattern 1 - Retry Logic with Exponential Backoff

The most common error handling pattern: retry transient failures with increasing delays.

### Flow
```
Execute → Success ✓
       → Failure → Wait → Retry → Success ✓
                        → Failure → Wait (2x) → Retry → Success ✓
                                              → Failure → Give up ✗
```

### Key Concepts
- **Exponential Backoff** - Double wait time after each retry
- **Max Retries** - Limit retry attempts to prevent infinite loops
- **Retry-able Errors** - Distinguish transient vs permanent failures
- **Jitter** - Add randomness to prevent thundering herd

In [2]:
async def retry_logic_demo():
    """
    Demonstrate retry logic with exponential backoff.
    Simulates API failures and automatic recovery.
    """
    print("="*70)
    print("PATTERN 1: Retry Logic with Exponential Backoff")
    print("="*70)
    print("\nSimulating API with random failures...\n")
    
    # Simulated unreliable executor
    class UnreliableExecutor(Executor):
        """Simulates random failures to demonstrate retry logic"""
        
        def __init__(self, failure_rate=0.6):
            super().__init__(id="unreliable_service")
            self.failure_rate = failure_rate
            self.attempt_count = 0
        
        @handler
        async def process(self, request: str, ctx: WorkflowContext[str, str]) -> None:
            self.attempt_count += 1
            
            # Simulate random failure
            if random.random() < self.failure_rate:
                error_msg = f" Attempt {self.attempt_count}: Service temporarily unavailable"
                print(error_msg)
                raise RuntimeError("Service unavailable - simulated failure")
            else:
                result = f" Attempt {self.attempt_count}: SUCCESS! Processed: {request}"
                print(result)
                await ctx.yield_output(result)
    
    # Retry wrapper executor
    class RetryExecutor(Executor):
        """Wraps another executor with retry logic"""
        
        def __init__(
            self, 
            inner_executor: Executor,
            max_retries: int = 3,
            initial_delay: float = 1.0,
            backoff_factor: float = 2.0,
            add_jitter: bool = True
        ):
            super().__init__(id=f"retry_{inner_executor.id}")
            self.inner_executor = inner_executor
            self.max_retries = max_retries
            self.initial_delay = initial_delay
            self.backoff_factor = backoff_factor
            self.add_jitter = add_jitter
        
        @handler
        async def execute_with_retry(self, request: str, ctx: WorkflowContext[str, str]) -> None:
            """Execute with exponential backoff retry logic"""
            
            delay = self.initial_delay
            last_error = None
            
            for attempt in range(self.max_retries + 1):
                try:
                    print(f"\n Retry attempt {attempt + 1}/{self.max_retries + 1}")
                    
                    # Try to execute
                    await self.inner_executor.process(request, ctx)
                    return  # Success!
                    
                except Exception as e:
                    last_error = e
                    
                    if attempt < self.max_retries:
                        # Calculate wait time with optional jitter
                        wait_time = delay
                        if self.add_jitter:
                            # Add random jitter (0-50% of delay)
                            jitter = random.uniform(0, delay * 0.5)
                            wait_time += jitter
                        
                        print(f"⏳ Waiting {wait_time:.2f}s before retry...")
                        await asyncio.sleep(wait_time)
                        
                        # Exponential backoff
                        delay *= self.backoff_factor
                    else:
                        # Max retries exceeded
                        print(f"\n💥 All {self.max_retries + 1} attempts failed!")
                        error_result = f" FAILED after {self.max_retries + 1} attempts: {str(last_error)}"
                        await ctx.yield_output(error_result)
    
    # Test the retry logic
    unreliable = UnreliableExecutor(failure_rate=0.6)  # 60% failure rate
    retry_wrapper = RetryExecutor(
        unreliable,
        max_retries=3,
        initial_delay=0.5,  # Start with 0.5s
        backoff_factor=2.0,  # Double each time: 0.5s, 1s, 2s
        add_jitter=True
    )
    
    workflow = (
        WorkflowBuilder()
        .set_start_executor(retry_wrapper)
        .build()
    )
    
    # Run test
    request = "Process important data"
    print(f" Request: {request}\n")
    
    final_output = None
    async for event in workflow.run_stream(request):
        if isinstance(event, WorkflowOutputEvent):
            final_output = event.data
    
    print("\n" + "="*70)
    print(" FINAL RESULT")
    print("="*70)
    if final_output:
        print(final_output)
    print("="*70)
    
    print("\n What Happened:")
    print("  1⃣  Service failed randomly (60% failure rate)")
    print("  2⃣  Retry logic kicked in with exponential backoff")
    print("  3⃣  Delays: 0.5s → 1.0s → 2.0s (+ jitter)")
    print("  4⃣  Either succeeded on retry or failed after max attempts")
    print("\n Production Settings:")
    print("  • API rate limits: 3-5 retries, 1s initial, 2x backoff")
    print("  • Network timeouts: 2-3 retries, 0.5s initial, 1.5x backoff")
    print("  • Always add jitter to prevent thundering herd")

await retry_logic_demo()

PATTERN 1: Retry Logic with Exponential Backoff

Simulating API with random failures...

 Request: Process important data


 Retry attempt 1/4
 Attempt 1: SUCCESS! Processed: Process important data

 FINAL RESULT
 Attempt 1: SUCCESS! Processed: Process important data

 What Happened:
  1⃣  Service failed randomly (60% failure rate)
  2⃣  Retry logic kicked in with exponential backoff
  3⃣  Delays: 0.5s → 1.0s → 2.0s (+ jitter)
  4⃣  Either succeeded on retry or failed after max attempts

 Production Settings:
  • API rate limits: 3-5 retries, 1s initial, 2x backoff
  • Network timeouts: 2-3 retries, 0.5s initial, 1.5x backoff
  • Always add jitter to prevent thundering herd


## Step 3: Pattern 2 - Fallback Strategies

When primary service fails, fall back to alternative approach.

### Flow
```
Try Primary Service → Success ✓
                    → Failure → Try Fallback → Success ✓
                                             → Failure → Error ✗
```

### Key Concepts
- **Primary/Secondary** - Preferred vs backup services
- **Graceful Degradation** - Lower quality but still useful
- **Cached Responses** - Return stale data when fresh unavailable
- **Simplified Operations** - Use simpler model or approach

In [ ]:
async def fallback_strategy_demo():
    """
    Demonstrate fallback to simpler approach when primary fails.
    Example: GPT-4 → GPT-3.5 → Cached response
    """
    print("="*70)
    print("PATTERN 2: Fallback Strategies")
    print("="*70)
    print("\nTiered fallback: Advanced Model → Simple Model → Cache\n")
    
    # Simulated cache
    RESPONSE_CACHE = {
        "weather": "Cached: Sunny, 72°F (last updated 1 hour ago)",
        "news": "Cached: Top headlines from this morning",
    }
    
    # Fallback executor with multiple tiers
    class FallbackExecutor(Executor):
        """Try multiple strategies in order until one succeeds"""
        
        def __init__(self):
            super().__init__(id="fallback_executor")
            self.primary_failures = 0
            self.fallback_successes = 0
            self.cache_hits = 0
        
        async def try_advanced_model(self, request: str) -> str:
            """Simulate advanced model (expensive, sometimes fails)"""
            print("   Trying advanced model (GPT-4)...")
            
            # Simulate 50% failure rate for demo
            if random.random() < 0.5:
                raise RuntimeError("Advanced model unavailable (rate limited)")
            
            return f" GPT-4 Response: Detailed analysis of '{request}'"
        
        async def try_simple_model(self, request: str) -> str:
            """Simulate simple model (cheaper, more reliable)"""
            print("   Falling back to simple model (GPT-3.5)...")
            
            # Simulate 20% failure rate
            if random.random() < 0.2:
                raise RuntimeError("Simple model also unavailable")
            
            return f" GPT-3.5 Response: Basic answer for '{request}'"
        
        async def try_cache(self, request: str) -> str:
            """Try cached response"""
            print("  📦 Checking cache...")
            
            # Simple keyword matching
            for key in RESPONSE_CACHE:
                if key in request.lower():
                    return f" {RESPONSE_CACHE[key]}"
            
            raise RuntimeError("No cached response available")
        
        @handler
        async def execute_with_fallback(self, request: str, ctx: WorkflowContext[str, str]) -> None:
            """Try strategies in order: advanced → simple → cache"""
            
            strategies = [
                ("Advanced Model", self.try_advanced_model),
                ("Simple Model", self.try_simple_model),
                ("Cache", self.try_cache),
            ]
            
            print(f" Request: {request}\n")
            
            for strategy_name, strategy_func in strategies:
                try:
                    print(f" Strategy: {strategy_name}")
                    result = await strategy_func(request)
                    
                    # Track metrics
                    if strategy_name == "Simple Model":
                        self.fallback_successes += 1
                    elif strategy_name == "Cache":
                        self.cache_hits += 1
                    
                    await ctx.yield_output(result)
                    return  # Success!
                    
                except Exception as e:
                    print(f"   {strategy_name} failed: {e}")
                    
                    if strategy_name == "Advanced Model":
                        self.primary_failures += 1
                    
                    # Try next strategy
                    continue
            
            # All strategies failed
            error_msg = " All fallback strategies exhausted - no response available"
            print(f"\n💥 {error_msg}")
            await ctx.yield_output(error_msg)
    
    # Build workflow
    fallback_executor = FallbackExecutor()
    
    workflow = (
        WorkflowBuilder()
        .set_start_executor(fallback_executor)
        .build()
    )
    
    # Test with multiple requests
    test_requests = [
        "What's the weather like?",
        "Summarize the latest tech news",
        "Explain quantum computing",  # Not in cache
    ]
    
    print("Testing fallback strategies with multiple requests...\n")
    
    for i, request in enumerate(test_requests, 1):
        print("\n" + "#"*70)
        print(f"TEST {i}")
        print("#"*70 + "\n")
        
        final_output = None
        async for event in workflow.run_stream(request):
            if isinstance(event, WorkflowOutputEvent):
                final_output = event.data
        
        print("\n" + "="*70)
        print(" RESULT")
        print("="*70)
        if final_output:
            print(final_output)
        print("="*70)
        
        # Small delay between tests
        await asyncio.sleep(0.5)
    
    # Show metrics
    print("\n" + "="*70)
    print(" METRICS")
    print("="*70)
    print(f"Primary failures: {fallback_executor.primary_failures}")
    print(f"Fallback successes: {fallback_executor.fallback_successes}")
    print(f"Cache hits: {fallback_executor.cache_hits}")
    print("="*70)
    
    print("\n What Happened:")
    print("  1⃣  Tried advanced model first (best quality)")
    print("  2⃣  On failure, fell back to simple model (good enough)")
    print("  3⃣  On failure, fell back to cache (stale but instant)")
    print("  4⃣  System degraded gracefully instead of failing completely")
    print("\n Production Use Cases:")
    print("  • Primary DB → Replica → Cache")
    print("  • Live API → Historical data → Default message")
    print("  • Real-time → Batch processed → Static content")

await fallback_strategy_demo()

## Step 4: Pattern 3 - Circuit Breaker

Prevent cascading failures by stopping requests to failing services.

### Flow
```
Closed (Normal) → Request → Success → Stay Closed
                → Request → Fail (threshold) → Open Circuit

Open (Blocking) → Wait (cooldown) → Half-Open

Half-Open (Testing) → Request → Success → Close Circuit
                    → Request → Fail → Stay Open
```

### Key Concepts
- **Failure Threshold** - Open circuit after N consecutive failures
- **Cooldown Period** - Wait before testing recovery
- **Half-Open State** - Allow test request to check if service recovered
- **Fast Fail** - Immediately reject when circuit open (don't waste time)

In [ ]:
async def circuit_breaker_demo():
    """
    Demonstrate circuit breaker pattern to prevent cascading failures.
    """
    print("="*70)
    print("PATTERN 3: Circuit Breaker")
    print("="*70)
    print("\nStates: Closed (OK) → Open (Failing) → Half-Open (Testing)\n")
    
    # Circuit breaker states
    class CircuitState(Enum):
        CLOSED = "CLOSED"        # Normal operation
        OPEN = "OPEN"            # Failing, reject immediately
        HALF_OPEN = "HALF_OPEN"  # Testing recovery
    
    # Circuit breaker executor
    class CircuitBreakerExecutor(Executor):
        """Prevents cascading failures by opening circuit on repeated failures"""
        
        def __init__(
            self,
            failure_threshold: int = 3,
            recovery_timeout: float = 5.0,
            simulate_recovery: bool = True
        ):
            super().__init__(id="circuit_breaker")
            self.failure_threshold = failure_threshold
            self.recovery_timeout = recovery_timeout
            self.simulate_recovery = simulate_recovery
            
            # State tracking
            self.state = CircuitState.CLOSED
            self.failure_count = 0
            self.last_failure_time: Optional[datetime] = None
            self.total_requests = 0
            self.rejected_requests = 0
            
            # Simulate service that fails then recovers
            self.service_is_down = True
            self.requests_until_recovery = 5
        
        def should_allow_request(self) -> bool:
            """Check if request should be allowed based on circuit state"""
            
            if self.state == CircuitState.CLOSED:
                return True
            
            elif self.state == CircuitState.OPEN:
                # Check if enough time has passed to try recovery
                if self.last_failure_time:
                    time_since_failure = datetime.now() - self.last_failure_time
                    if time_since_failure.total_seconds() >= self.recovery_timeout:
                        print("   Circuit moving to HALF-OPEN (testing recovery)")
                        self.state = CircuitState.HALF_OPEN
                        return True
                
                # Still in cooldown
                return False
            
            elif self.state == CircuitState.HALF_OPEN:
                # Allow single test request
                return True
            
            return False
        
        async def call_service(self, request: str) -> str:
            """Simulate calling external service"""
            
            # Simulate gradual recovery
            if self.simulate_recovery:
                self.requests_until_recovery -= 1
                if self.requests_until_recovery <= 0:
                    self.service_is_down = False
            
            if self.service_is_down:
                raise RuntimeError("External service is down")
            
            return f" Service response: Processed '{request}'"
        
        def record_success(self):
            """Record successful request"""
            print("   Request succeeded")
            self.failure_count = 0
            
            if self.state == CircuitState.HALF_OPEN:
                print("  🔓 Circuit CLOSED (service recovered)")
                self.state = CircuitState.CLOSED
        
        def record_failure(self):
            """Record failed request"""
            print("   Request failed")
            self.failure_count += 1
            self.last_failure_time = datetime.now()
            
            if self.failure_count >= self.failure_threshold:
                if self.state != CircuitState.OPEN:
                    print(f"  🚨 Circuit OPEN (threshold {self.failure_threshold} reached)")
                    self.state = CircuitState.OPEN
            
            if self.state == CircuitState.HALF_OPEN:
                print("  🔒 Circuit back to OPEN (recovery test failed)")
                self.state = CircuitState.OPEN
        
        @handler
        async def execute(self, request: str, ctx: WorkflowContext[str, str]) -> None:
            """Execute with circuit breaker protection"""
            
            self.total_requests += 1
            
            # Check circuit state
            if not self.should_allow_request():
                self.rejected_requests += 1
                remaining = self.recovery_timeout - (datetime.now() - self.last_failure_time).total_seconds()
                
                result = (
                    f"⚡ CIRCUIT OPEN - Request rejected (fast fail)\n"
                    f"   Service is unhealthy. Try again in {remaining:.1f}s"
                )
                print(f"  {result}")
                await ctx.yield_output(result)
                return
            
            # Try to execute
            try:
                result = await self.call_service(request)
                self.record_success()
                await ctx.yield_output(result)
                
            except Exception as e:
                self.record_failure()
                await ctx.yield_output(f" Service error: {e}")
        
        def get_metrics(self) -> dict:
            """Get circuit breaker metrics"""
            return {
                "state": self.state.value,
                "total_requests": self.total_requests,
                "rejected_requests": self.rejected_requests,
                "failure_count": self.failure_count,
                "rejection_rate": f"{(self.rejected_requests / max(self.total_requests, 1)) * 100:.1f}%"
            }
    
    # Build workflow
    circuit_breaker = CircuitBreakerExecutor(
        failure_threshold=3,
        recovery_timeout=3.0,  # 3 seconds
        simulate_recovery=True
    )
    
    workflow = (
        WorkflowBuilder()
        .set_start_executor(circuit_breaker)
        .build()
    )
    
    # Simulate multiple requests over time
    print("Simulating requests to failing service...\n")
    
    for i in range(10):
        print(f"\n{'='*70}")
        print(f"REQUEST {i + 1}/10 - Circuit State: {circuit_breaker.state.value}")
        print("="*70)
        
        final_output = None
        async for event in workflow.run_stream(f"Request #{i + 1}"):
            if isinstance(event, WorkflowOutputEvent):
                final_output = event.data
        
        if final_output:
            print(f"Result: {final_output}")
        
        # Small delay between requests
        await asyncio.sleep(0.5)
    
    # Show metrics
    metrics = circuit_breaker.get_metrics()
    print("\n" + "="*70)
    print(" CIRCUIT BREAKER METRICS")
    print("="*70)
    for key, value in metrics.items():
        print(f"{key}: {value}")
    print("="*70)
    
    print("\n What Happened:")
    print("  1⃣  Service failed repeatedly (first 3 requests)")
    print("  2⃣  Circuit opened after failure threshold")
    print("  3⃣  Subsequent requests rejected immediately (fast fail)")
    print("  4⃣  After cooldown, circuit moved to half-open")
    print("  5⃣  Test request succeeded → circuit closed")
    print("  6⃣  Normal operation resumed")
    print("\n Benefits:")
    print("  • Prevent wasting resources on failing service")
    print("  • Give service time to recover")
    print("  • Fast fail instead of long timeouts")
    print("  • Automatic recovery testing")

await circuit_breaker_demo()

## Step 5: Pattern 4 - Timeout Protection

Prevent operations from hanging indefinitely.

### Key Concepts
- **Operation Timeout** - Max time for single operation
- **Workflow Timeout** - Max time for entire workflow
- **Cancellation** - Properly stop long-running tasks
- **Cleanup** - Release resources on timeout

In [ ]:
async def timeout_protection_demo():
    """
    Demonstrate timeout protection for long-running operations.
    """
    print("="*70)
    print("PATTERN 4: Timeout Protection")
    print("="*70)
    print("\nPrevent operations from hanging indefinitely\n")
    
    # Executor with built-in timeout
    class TimeoutExecutor(Executor):
        """Wraps operations with timeout protection"""
        
        def __init__(self, timeout_seconds: float = 5.0):
            super().__init__(id="timeout_executor")
            self.timeout_seconds = timeout_seconds
            self.timed_out_count = 0
            self.completed_count = 0
        
        async def slow_operation(self, duration: float, request: str) -> str:
            """Simulate slow operation"""
            print(f"  ⏳ Starting operation (will take {duration}s)...")
            await asyncio.sleep(duration)
            return f" Completed: {request}"
        
        @handler
        async def execute_with_timeout(self, request: str, ctx: WorkflowContext[str, str]) -> None:
            """Execute with timeout protection"""
            
            # Parse duration from request (for demo)
            try:
                duration = float(request.split(":")[-1].strip().replace("s", ""))
            except:
                duration = 3.0
            
            print(f" Request: {request}")
            print(f"⏰ Timeout: {self.timeout_seconds}s")
            
            try:
                # Use asyncio.wait_for for timeout
                result = await asyncio.wait_for(
                    self.slow_operation(duration, request),
                    timeout=self.timeout_seconds
                )
                
                self.completed_count += 1
                print(f"  {result}")
                await ctx.yield_output(result)
                
            except asyncio.TimeoutError:
                self.timed_out_count += 1
                timeout_msg = (
                    f"⏰ TIMEOUT after {self.timeout_seconds}s\n"
                    f"   Operation was taking too long and was cancelled"
                )
                print(f"  {timeout_msg}")
                await ctx.yield_output(timeout_msg)
        
        def get_metrics(self) -> dict:
            total = self.completed_count + self.timed_out_count
            return {
                "completed": self.completed_count,
                "timed_out": self.timed_out_count,
                "timeout_rate": f"{(self.timed_out_count / max(total, 1)) * 100:.1f}%"
            }
    
    # Build workflow
    timeout_executor = TimeoutExecutor(timeout_seconds=3.0)
    
    workflow = (
        WorkflowBuilder()
        .set_start_executor(timeout_executor)
        .build()
    )
    
    # Test with various durations
    test_requests = [
        "Fast operation: 1s",
        "Medium operation: 2.5s",
        "Slow operation: 4s",  # Will timeout
        "Very slow operation: 6s",  # Will timeout
        "Quick operation: 0.5s",
    ]
    
    print("Testing timeout protection...\n")
    
    for i, request in enumerate(test_requests, 1):
        print(f"\n{'='*70}")
        print(f"TEST {i}/{len(test_requests)}")
        print("="*70)
        
        final_output = None
        async for event in workflow.run_stream(request):
            if isinstance(event, WorkflowOutputEvent):
                final_output = event.data
        
        print(f"\n Result: {final_output}")
    
    # Show metrics
    metrics = timeout_executor.get_metrics()
    print("\n" + "="*70)
    print(" TIMEOUT METRICS")
    print("="*70)
    for key, value in metrics.items():
        print(f"{key}: {value}")
    print("="*70)
    
    print("\n What Happened:")
    print("  1⃣  Operations under 3s completed successfully")
    print("  2⃣  Operations over 3s were cancelled (timeout)")
    print("  3⃣  System didn't hang waiting for slow operations")
    print("  4⃣  Resources released immediately on timeout")
    print("\n Production Settings:")
    print("  • API calls: 10-30s timeout")
    print("  • Database queries: 5-10s timeout")
    print("  • Agent responses: 30-60s timeout")
    print("  • Entire workflow: 5-10 minutes timeout")

await timeout_protection_demo()

##  Key Takeaways

### What We Learned

1. **Retry Logic**
   - Exponential backoff: 1s → 2s → 4s
   - Add jitter to prevent thundering herd
   - Distinguish transient vs permanent failures
   - Set max retries to prevent infinite loops

2. **Fallback Strategies**
   - Primary → Secondary → Tertiary
   - Graceful degradation (good enough > nothing)
   - Cache stale data as last resort
   - Track which tier served the request

3. **Circuit Breaker**
   - Three states: Closed → Open → Half-Open
   - Fast fail when circuit open (save resources)
   - Automatic recovery testing
   - Prevent cascading failures

4. **Timeout Protection**
   - Set reasonable timeouts for all operations
   - Use asyncio.wait_for() for async operations
   - Clean up resources on timeout
   - Don't let system hang indefinitely

### Error Handling Decision Tree

```
Error Occurs
├─ Is it transient? (network, rate limit)
│  └─ YES → Retry with exponential backoff
│
├─ Can we use fallback? (cached, simpler)
│  └─ YES → Try fallback strategy
│
├─ Is service repeatedly failing?
│  └─ YES → Open circuit breaker
│
├─ Is operation hanging?
│  └─ YES → Apply timeout
│
└─ None apply → Fail fast with clear error
```

### Production Patterns

```python
# Retry with exponential backoff
async def retry_with_backoff(func, max_retries=3):
    delay = 1.0
    for attempt in range(max_retries):
        try:
            return await func()
        except TransientError:
            if attempt < max_retries - 1:
                await asyncio.sleep(delay + random.uniform(0, delay * 0.5))
                delay *= 2
            else:
                raise

# Fallback chain
async def execute_with_fallback(request):
    try:
        return await primary_service(request)
    except:
        try:
            return await fallback_service(request)
        except:
            return get_cached_response(request)

# Timeout protection
async def execute_with_timeout(func, timeout=30.0):
    try:
        return await asyncio.wait_for(func(), timeout=timeout)
    except asyncio.TimeoutError:
        raise OperationTimeoutError(f"Exceeded {timeout}s")
```

### Combining Patterns

In production, combine multiple patterns:

```python
# Retry + Fallback + Timeout
async def robust_execute(request):
    # 1. Try with timeout
    try:
        return await asyncio.wait_for(
            retry_with_backoff(lambda: primary_service(request)),
            timeout=30.0
        )
    except (TimeoutError, MaxRetriesExceeded):
        # 2. Fall back to simpler approach
        return await fallback_service(request)
```

---

##  Practice Exercises

### Exercise 1: Smart Retry

Build a retry executor that:
- Only retries specific error types (not all errors)
- Tracks retry success rate
- Adjusts retry strategy based on error patterns

### Exercise 2: Multi-Tier Fallback

Create a fallback chain:
- Primary: Real-time API
- Secondary: 5-minute cached data
- Tertiary: Daily aggregated data
- Final: Default static response

### Exercise 3: Adaptive Circuit Breaker

Build a circuit breaker that:
- Adjusts thresholds based on error rate trends
- Has different thresholds for different error types
- Provides detailed health metrics

### Exercise 4: Comprehensive Error Handler

Combine all patterns:
- Retry transient failures
- Fall back on persistent failures
- Open circuit on cascading failures
- Apply timeouts to all operations
- Log all errors for monitoring

In [ ]:
# Exercise playground - implement your solutions here!

async def smart_retry_exercise():
    """
    Exercise 1: Build retry with error type filtering
    """
    # TODO: Implement selective retry logic
    pass

async def multi_tier_fallback_exercise():
    """
    Exercise 2: Build multi-tier fallback chain
    """
    # TODO: Implement 4-tier fallback
    pass

async def adaptive_circuit_breaker_exercise():
    """
    Exercise 3: Build adaptive circuit breaker
    """
    # TODO: Implement dynamic thresholds
    pass

async def comprehensive_error_handler_exercise():
    """
    Exercise 4: Combine all error handling patterns
    """
    # TODO: Implement combined error handling
    pass

print(" Exercise templates ready - implement your solutions above!")

##  What's Next?

Congratulations! You've mastered error handling and recovery patterns!

You now know how to:
-  Implement retry logic with exponential backoff
-  Build fallback strategies for graceful degradation
-  Use circuit breakers to prevent cascading failures
-  Apply timeout protection to prevent hangs
-  Combine patterns for robust production systems

**Coming in Future Tutorials:**
- **Tutorial 10: Monitoring & Observability** - Logging, metrics, tracing, debugging
- **Tutorial 11: Testing AI Workflows** - Unit tests, integration tests, mocking
- **Tutorial 12: Production Deployment** - Best practices, security, scaling, CI/CD

---

### Quick Reference

**Retry Pattern:**
```python
for attempt in range(max_retries):
    try:
        return await operation()
    except TransientError:
        await asyncio.sleep(delay)
        delay *= 2  # Exponential backoff
```

**Fallback Pattern:**
```python
try:
    return await primary()
except:
    try:
        return await secondary()
    except:
        return cached_response()
```

**Circuit Breaker Pattern:**
```python
if circuit_open:
    raise ServiceUnavailable("Circuit open")
try:
    result = await service()
    close_circuit()
    return result
except:
    open_circuit()
    raise
```

**Timeout Pattern:**
```python
try:
    return await asyncio.wait_for(operation(), timeout=30)
except asyncio.TimeoutError:
    raise OperationTimeout()
```

---

** Great job completing Tutorial 09!**